In [34]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/ljgoico/desktop/ironhack/Ironhack Data Analytics.json"

from google.cloud import bigquery

client = bigquery.Client()

## Challenge 1 - Most profiting authors

#### Step 1

In [35]:
QUERY = '''
SELECT
    titles.title_id,
    authors.au_id,
    titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100 AS royalty_per_sale_au
    
FROM `ironhack-data-analytics.publications.authors` authors


JOIN
    `ironhack-data-analytics.publications.titleauthor` titleauthor
ON
    authors.au_id = titleauthor.au_id
    
INNER JOIN
    `ironhack-data-analytics.publications.sales` sales
ON
    titleauthor.title_id = sales.title_id
    
INNER JOIN
    `ironhack-data-analytics.publications.titles` titles
ON
    sales.title_id = titles.title_id

ORDER BY royalty_per_sale_au DESC

'''

In [36]:
query_job = client.query(QUERY)

df = query_job.to_dataframe()

df.head(10)

,title_id,au_id,royalty_per_sale_au
0,PC1035,238-95-7766,110.160
1,TC3218,807-91-6654,83.800
2,PC8888,846-92-7186,50.000
3,PC8888,427-17-2319,50.000
4,PS2091,899-46-2035,49.275
5,PS2091,998-72-3567,49.275
6,TC4203,648-92-1872,33.460
7,PS1372,756-30-7391,32.385
8,PS3333,172-32-1176,29.985
9,BU7832,274-80-9391,29.985


#### Step 2

In [37]:
QUERY2 = '''
SELECT
    titles.title_id,
    authors.au_id,
    SUM(titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) AS total_royalties_title_au
    
FROM `ironhack-data-analytics.publications.authors` authors


JOIN
    `ironhack-data-analytics.publications.titleauthor` titleauthor
ON
    authors.au_id = titleauthor.au_id
    
INNER JOIN
    `ironhack-data-analytics.publications.sales` sales
ON
    titleauthor.title_id = sales.title_id
    
INNER JOIN
    `ironhack-data-analytics.publications.titles` titles
ON
    sales.title_id = titles.title_id
    
GROUP BY 
    au_id,
    title_id

ORDER BY total_royalties_title_au DESC

'''

In [38]:
query_job = client.query(QUERY2)

df2 = query_job.to_dataframe()

df2.head(10)

,title_id,au_id,total_royalties_title_au
0,PC1035,238-95-7766,110.160
1,TC3218,807-91-6654,83.800
2,PS2091,899-46-2035,70.956
3,PS2091,998-72-3567,70.956
4,PC8888,427-17-2319,50.000
5,PC8888,846-92-7186,50.000
6,TC4203,648-92-1872,33.460
7,PS1372,756-30-7391,32.385
8,BU7832,274-80-9391,29.985
9,PS3333,172-32-1176,29.985


#### Step 3

In [44]:
QUERY3 = '''
SELECT
    authors.au_id,
    SUM(titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) + titles.advance AS PROFITS
    
FROM `ironhack-data-analytics.publications.authors` authors


JOIN
    `ironhack-data-analytics.publications.titleauthor` titleauthor
ON
    authors.au_id = titleauthor.au_id
    
INNER JOIN
    `ironhack-data-analytics.publications.sales` sales
ON
    titleauthor.title_id = sales.title_id
    
INNER JOIN
    `ironhack-data-analytics.publications.titles` titles
ON
    sales.title_id = titles.title_id
    
GROUP BY 
    authors.au_id,
    titles.advance

ORDER BY PROFITS DESC

LIMIT 3

'''

In [45]:
query_job = client.query(QUERY3)

df3 = query_job.to_dataframe()

df3.head()

,au_id,PROFITS
0,722-51-5454,15021.528
1,899-46-2035,15007.176
2,213-46-8915,10150.116


## Challenge 2 - Create a table

In [53]:
QUERY4 = '''

CREATE TABLE `ironhack-data-analytics.publications.most_profiting_authors_laura`

SELECT
    authors.au_id,
    SUM(titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) + titles.advance AS PROFITS
    
FROM `ironhack-data-analytics.publications.authors` authors


JOIN
    `ironhack-data-analytics.publications.titleauthor` titleauthor
ON
    authors.au_id = titleauthor.au_id
    
INNER JOIN
    `ironhack-data-analytics.publications.sales` sales
ON
    titleauthor.title_id = sales.title_id
    
INNER JOIN
    `ironhack-data-analytics.publications.titles` titles
ON
    sales.title_id = titles.title_id
    
GROUP BY 
    authors.au_id,
    titles.advance

ORDER BY PROFITS DESC

'''

In [51]:
QUERY5 = '''
SELECT *
FROM publications.most_profiting_authors_laura'''

In [54]:
query_job = client.query(QUERY4)

df4 = query_job.to_dataframe()

df4.head(5)

BadRequest: 400 Syntax error: Unexpected keyword SELECT at [5:1]